In [1]:
import pandas as pd
import numpy as np

D:\ProgramData\Anaconda3\envs\study_py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramData\Anaconda3\envs\study_py37\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\ProgramData\Anaconda3\envs\study_py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


# Relief
$$\delta^j = \sum_i -diff(x_i^j, x_{i,nh}^j)^2 + diff(x_i^j, x_{i, nm}^j)^2$$

其中$x_a^j$表示样本$x_a$在属性$j$上的取值，$diff(x_a^j, x_b^j)$取决于属性$j$的类型：离散型，相同为1不同为0；连续型，$diff(x_a^j, x_b^j)=\vert x_a^j - x_b^j\vert$,$x_a^j, x_b^j$需要规范化到$[0, 1]$区间.

$x_{i,nh}$称为猜中近邻(near-hit)，是同类样本中的最近邻，$x_{i, nm}$称为猜错近邻(near-miss)，异类样本中的最近邻。

In [4]:
class Relief:
    def readData(self):
        dataset = pd.read_csv('./data_watermelon/watermelon3_0_Ch.csv')  # 读取数据
        self.Attributes = dataset.columns[1:-1]        #所有属性的名称
        #print(self.Attributes)
        dataset = np.matrix(dataset)
        dataset = dataset[:,1:]
        m,n = np.shape(dataset)      # 得到数据集大小
        #print(dataset)
        dataset[:,n-3:n-1] = (dataset[:,n-3:n-1]-np.min(dataset[:,n-3:n-1],0))/\
                                (np.max(dataset[:,n-3:n-1],0)-np.min(dataset[:,n-3:n-1],0))     # 连续属性规范化
        self.goodindex = np.where(dataset[:,n-1]=='是')[0]        # 好瓜的索引
        self.badindex = np.where(dataset[:,n-1]=='否')[0]         # 坏瓜的索引
        self.dataset = dataset
    # 计算每个样本之间的距离
    def getDist(self):
        m,n = np.shape(self.dataset)
        distance = np.ones([m,m])
        for i in range(m):
            distance[i,i] = np.inf
            for j in range(i+1,m):
                dis = 0
                for k in range(n-1):
                    if type(self.dataset[i,k])== str:        # 离散属性
                        dis = dis + 1 - np.int(self.dataset[i,k]==self.dataset[j,k])
                    else:                                   # 连续属性
                        dis = dis + pow((self.dataset[i,k]-self.dataset[j,k]),2)
                distance[i,j] = dis
                distance[j,i] = dis
        self.distance = distance
 
    # 过滤
    def fliter(self):
        m,n = np.shape(self.dataset)
        delte = np.zeros([n-1])
        for i in range(m):    # 依次对每一个样本进行
            delte = delte + self.getdelet(i)
        return delte
 
    # 计算一个样本中所有属性的权值delte
    def getdelet(self,i):
        m,n = np.shape(self.dataset)
        distance = self.distance[i,:]            # 该样本i到每一个样本的距离
        gooddis = distance[self.goodindex]       # 到好瓜距离
        baddis = distance[self.badindex]         # 到坏瓜距离
        good = dict(zip(gooddis,self.goodindex))     # 生成字典，键为距离，值为样本索引
        bad = dict(zip(baddis,self.badindex))
        if self.dataset[i,-1] == '是':               # 样本i是好瓜
            minh = np.min(gooddis)       # 同类中最小距离
            indexh = good[minh]          # 最小距离对应的样本索引
            minm = np.min(baddis)        # 不同类中最小距离
            indexm = bad[minm]           # 不同类中最小距离对应的索引
        else:
            minh = np.min(baddis)       # 同类
            indexh = bad[minh]
            minm = np.min(gooddis)      # 不同类
            indexm = good[minm]
        delte = []
        for j in range(n-1):           # 遍历每一个属性，进行计算
            if type(self.dataset[i,j])== str:     # 离散属性
                dh = 1 - np.int(self.dataset[i,j]==self.dataset[indexh,j])     # 同类
                dm = 1 - np.int(self.dataset[i,j]==self.dataset[indexm,j])     # 不同类
            else:                                # 连续属性
                dh = np.float64(pow((self.dataset[i,j]-self.dataset[indexh,j]),2))   # 同类
                dm = np.float64(pow((self.dataset[i,j]-self.dataset[indexm,j]),2))   # 不同类
            delte.append(-dh+dm)
        return delte                    # 返回的是样本i计算得到的每一个属性上的权值
 
 
    def getAttr(self,delte):
        dic = dict(zip(self.Attributes,delte))       # 得到字典
        dd = sorted(dic.items(),key=lambda item:item[1],reverse=True)      # 根据值的大小进行排序，倒序
        for i,j in dd:
            print(i+':'+str(j),end='\t')

In [5]:
relief = Relief()
relief.readData()
relief.getDist()
delte = relief.fliter()
relief.getAttr(delte)

纹理:9.0	脐部:7.0	根蒂:4.0	含糖率:2.044916554108194	密度:-0.5009664457141232	敲声:-1.0	触感:-4.0	色泽:-7.0	

C:\Users\FX2\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\FX2\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:61: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note l